In [ ]:
!conda install lxml

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2s             |       h7b6447c_0         3.1 MB
    certifi-2019.6.16          |           py36_1         156 KB
    lxml-4.3.0                 |   py36hefd8a0e_0         1.5 MB
    ------------------------------------------------------------
                                           Total:         4.8 MB

The following packages will be UPDATED:

    certifi: 2019.6.16-py36_1     conda-forge --> 2019.6.16-py36_1    
    lxml:    4.2.5-py37hefd8a0e_0             --> 4.3.0-py36hefd8a0e_0
    openssl: 1.0.2

In [1]:
import pandas as pd
import html5lib
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)
#read_html returns a list of dataframe objects, so the first object in the list is the pandas dataframe. Use index 0 to access it. 

In [2]:
df = data[0]
df.shape

(288, 3)

<h3> Drop any values that are 'Not assigned' in the Borough column </h3>

In [3]:
df_1 = df[df.Borough != 'Not assigned']
df_1.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


<h3> Check to make sure the drop worked </h3>

In [4]:
df_1.loc[df_1.Neighbourhood == 'Not assigned']

,Postcode,Borough,Neighbourhood
8,M7A,Queen's Park,Not assigned


<h3> Replace 'Not assigned' value in the Neighbourhood column with the correspoding Borough value </h3>

In [5]:
df_1['Neighbourhood'].replace('Not assigned', value=(df_1.loc[8, 'Borough']))

2                     Parkwoods
3              Victoria Village
4                  Harbourfront
5                   Regent Park
6              Lawrence Heights
                 ...           
282    Kingsway Park South West
283                   Mimico NW
284          The Queensway West
285       Royal York South West
286              South of Bloor
Name: Neighbourhood, Length: 211, dtype: object

In [6]:
df_1.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


<h3> Group by Postcode and aggregate Neighbourhood values </h3>

In [23]:
df_2 = df_1.groupby('Postal Code', as_index='False').agg(lambda x : ','.join(set(x)))
df_2.head()

,Borough,Neighbourhood
Postal Code,,
M1B,Scarborough,"Malvern,Rouge"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Morningside,Guildwood,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [8]:
df_2.shape

(103, 2)

<h3> Add coordinates </h3>

In [9]:
Geo = pd.read_csv('data/Geospatial_Coordinates.csv')
Geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
df_2.rename(columns={"Postcode": "Postal Code"}, inplace=True)
df_2.head()

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Malvern,Rouge"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Morningside,Guildwood,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [17]:
pd.merge(df_2, Geo, on='Postal Code')

KeyError: 'Postal Code'